<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>


# LinkedIn - Accept all LinkedIn invitations and send first message

**Tags:** #linkedin #content #operations #automation #invitation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

In this notebook, you will be able to automically accept all invitations from LinkedIn

## Input

### Import libraries


In [1]:
from naas_drivers import linkedin
import naas
import pandas as pd

### Setup LinkedIn

- [Get your cookies](/d20a8e7e508e42af8a5b52e33f3dba75)

In [2]:
# Lindekin cookies
LI_AT = "AQEDARCNSioDe6wmAAABfqF-HR4AAAF-xYqhHlYAtSu7EZZEpFer0UZF-GLuz2DNSz4asOOyCRxPGFjenv37irMObYYgxxxxxxx"
JSESSIONID = "ajax:12XXXXXXXXXXXXXXXXX"

# First message
FIRST_MESSAGE = "Hello, Nice to connect!"

In [3]:
LI_AT = naas.secret.get("LI_AT")
JSESSIONID = naas.secret.get("JSESSIONID")

### Setup Naas

In [ ]:
# Schedule your notebook every hour
naas.scheduler.add(cron="0 * * * *")

#-> Uncomment the line below to remove your scheduler
# naas.scheduler.delete()

## Model

### Get pending invitations

In [4]:
df_pending = linkedin.connect(LI_AT, JSESSIONID).invitation.get()
df_pending

,PROFILE_ID,PUBLIC_ID,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,PROFILE_PICTURE,MESSAGE,SENT_AT,UNSEEN,INVITATION_TYPE,INVITATION_DESC,INVITATION_STATUS,INVITATION_ID,SHARED_SECRET
0,ACoAAB45mmMBZTKv30NKmXayIB3WzBGe75lDNB0,muhtasham,Muhtasham,Oblokulov,Muhtasham Oblokulov,NLP at Alyne | MSc at TUM | Multi-Task Learnin...,https://media-exp1.licdn.com/dms/image/C4E03AQ...,None,2022-04-02 10:13:06,False,Profile,Invitation • 2022-04-02 10:13:06,PENDING,6915964281721098240,6xubL2KM


In [10]:
linkedin.connect(LI_AT, JSESSIONID).invitation.ignore("6916726902791561266", "Fe4OlQxY", True)

❌ Invitation ignored !
❌ Invitation ignored !


""


In [19]:
linkedin.connect(LI_AT, JSESSIONID).message.send("ACoAAB45mmMBZTKv30NKmXayIB3WzBGe75lDNB0", FIRST_MESSAGE)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Accept all pending invitations and send first message

In [17]:
def accept_new_contact(df):
    df_accept = pd.DataFrame()
    
    # Loop
    for index, row in df.iterrows():
        status = row.INVITATION_STATUS
        invitation_id = row.INVITATION_ID
        shared_secret = row.SHARED_SECRET
        profile_id = row.PROFILE_ID
        if status == "PENDING":
            tmp_df = linkedin.connect(LI_AT, JSESSIONID).invitation.accept(invitation_id, shared_secret)
            df_accept = pd.concat([df_accept, tmp_df])
            linkedin.connect(LI_AT, JSESSIONID).message.send(profile_id, FIRST_MESSAGE)
    return df_accept

df_accept = accept_new_contact(df_pending)
df_accept

🤝 Invitation accepted !


TypeError: cannot concatenate object of type '<class 'requests.exceptions.HTTPError'>'; only Series and DataFrame objs are valid

## Output


### Display result


In [ ]:
df_accept